#Load the original data

In [ ]:
!unzip "/content/drive/MyDrive/Invigor.zip"


Archive:  /content/drive/MyDrive/Invigor.zip
  inflating: Invigor/Customer 1 Input List Consumer Electronics.xlsx  
  inflating: Invigor/customer 2 example matching.xlsx  
  inflating: Invigor/customer 2 matched example.xlsx  
  inflating: Invigor/matched CE example.xlsx  


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('/content/Invigor/matched CE example.xlsx')
df.head()

,skuId,productId,productDescription,skuName,department,brandName,modelNo,productName (BP),BrandName (BP),ModelNo (BP),Base Product ID
0,1650139,4757350,Garmin Vivosmart 4 Fitness Tracker S/M - Grey/...,GARMIN Vivosmart 4 Gray rose Gold Sm 010-01995-12,ELECTRONICS,Garmin,010-01995-12,Vivosmart 4 Activity Tracker Grey Rose Gold,GARMIN,010-01995-12,411105
1,1656981,5498218,"Garmin 5"" Drive 52 & Live Traffic GPS",GARMIN Drive 52 MT-S 010-02036-42,ELECTRONICS,Garmin,010-02036-42,Drive 52 GPS,GARMIN,010-02036-42,414413
2,1650182,3798078,Garmin 45mm Instinct Bluetooth GPS Sport Watch...,GARMIN Instinct Tundra 010-02064-01,ELECTRONICS,Garmin,010-02064-01,Instinct Sports Watch - Tundra,GARMIN,010-02064-01,411107
3,1650176,6251374,Garmin 30.4mm Forerunner 245 Music Smartwatch ...,GARMIN Forerunner 245 Music Black 010-02120-30,ELECTRONICS,Garmin,010-02120-30,Forerunner 245 Music Black,GARMIN,010-02120-30,409727
4,1650178,6397718,Garmin 30.4mm Forerunner 245 Music Fitness Sma...,GARMIN Forerunner 245 Music Aqua 010-02120-32,ELECTRONICS,Garmin,010-02120-32,Garmin Forerunner 245 Music Smart Watch - Aqua,GARMIN,010-02120-32,409129


# Separate the core fields

In [ ]:
df_correct = df[['productDescription','productName (BP)','BrandName (BP)','department']].copy()
df_correct.rename(columns={'productDescription': 'external_name', 
                   'productName (BP)': 'internal_name'}, inplace=True)
df_correct.head()

,external_name,internal_name,BrandName (BP),department
0,Garmin Vivosmart 4 Fitness Tracker S/M - Grey/...,Vivosmart 4 Activity Tracker Grey Rose Gold,GARMIN,ELECTRONICS
1,"Garmin 5"" Drive 52 & Live Traffic GPS",Drive 52 GPS,GARMIN,ELECTRONICS
2,Garmin 45mm Instinct Bluetooth GPS Sport Watch...,Instinct Sports Watch - Tundra,GARMIN,ELECTRONICS
3,Garmin 30.4mm Forerunner 245 Music Smartwatch ...,Forerunner 245 Music Black,GARMIN,ELECTRONICS
4,Garmin 30.4mm Forerunner 245 Music Fitness Sma...,Garmin Forerunner 245 Music Smart Watch - Aqua,GARMIN,ELECTRONICS


In [ ]:
df_correct['BrandName (BP)'].isnull().sum()
df_correct['internal_name'].isnull().sum()


0

In [ ]:
df_correct.dtypes

external_name     object
internal_name     object
BrandName (BP)    object
department        object
dtype: object

#Label the correct predictions

In [ ]:
df_correct['match'] = 1
df_correct.head()


,external_name,internal_name,BrandName (BP),department,match
0,Garmin Vivosmart 4 Fitness Tracker S/M - Grey/...,Vivosmart 4 Activity Tracker Grey Rose Gold,GARMIN,ELECTRONICS,1
1,"Garmin 5"" Drive 52 & Live Traffic GPS",Drive 52 GPS,GARMIN,ELECTRONICS,1
2,Garmin 45mm Instinct Bluetooth GPS Sport Watch...,Instinct Sports Watch - Tundra,GARMIN,ELECTRONICS,1
3,Garmin 30.4mm Forerunner 245 Music Smartwatch ...,Forerunner 245 Music Black,GARMIN,ELECTRONICS,1
4,Garmin 30.4mm Forerunner 245 Music Fitness Sma...,Garmin Forerunner 245 Music Smart Watch - Aqua,GARMIN,ELECTRONICS,1


In [ ]:
# Add the brand name to the internal_name 
# .astype(str) will deal with the integers in string
df_correct['internal_name']= df_correct['BrandName (BP)'].astype(str)+ " "+ df_correct['internal_name'].astype(str)

In [ ]:
df_correct.internal_name.nunique()

995

In [ ]:
df_correct.head()

,external_name,internal_name,BrandName (BP),department,match
0,Garmin Vivosmart 4 Fitness Tracker S/M - Grey/...,GARMIN Vivosmart 4 Activity Tracker Grey Rose ...,GARMIN,ELECTRONICS,1
1,"Garmin 5"" Drive 52 & Live Traffic GPS",GARMIN Drive 52 GPS,GARMIN,ELECTRONICS,1
2,Garmin 45mm Instinct Bluetooth GPS Sport Watch...,GARMIN Instinct Sports Watch - Tundra,GARMIN,ELECTRONICS,1
3,Garmin 30.4mm Forerunner 245 Music Smartwatch ...,GARMIN Forerunner 245 Music Black,GARMIN,ELECTRONICS,1
4,Garmin 30.4mm Forerunner 245 Music Fitness Sma...,GARMIN Garmin Forerunner 245 Music Smart Watch...,GARMIN,ELECTRONICS,1


#Create incorrect data

In [ ]:
def create_synthetic_data(df, iterations):
    """Creates synthetic training data from the correctly matched
    data by grouping on the cluster_label column and reshuffling
    the internal_name to create data that contain incorrect matches.
    """

    df_output = df

    i = 1
    while i <= iterations:

        # Create synthetic data by shuffling the column using a groupby
        df_s = df[['external_name','internal_name','BrandName (BP)','department']].copy()
        df_s['shuffled_internal_name'] = df_s['internal_name']
        df_s['shuffled_internal_name'] = df_s.groupby('department')['internal_name'].transform(np.random.permutation)

        # Add the correct value to the match column
        df_s['match'] = np.where(df_s['internal_name'] == df_s['shuffled_internal_name'], 1, 0)

        # Create internal name column
        df_s['internal_name'] = np.where(df_s['shuffled_internal_name']!='', 
                                        df_s['shuffled_internal_name'],
                                        df_s['internal_name'])

        df_output = df_output.append(df_s)
        df_output = df_output.drop(columns=['shuffled_internal_name'])

        i += 1

    return df_output

In [ ]:
df_output = create_synthetic_data(df_correct, 10)
df_output.tail()

,external_name,internal_name,BrandName (BP),department,match
991,Electrolux Ergorapido 18V Stick Vacuum Cleaner...,SHARK Klik n' Flip Steam Pocket Mop s6001,ELECTROLUX,APPLIANCES,0
992,2 x Lenovo 7W Smart Filament Bulb ST64 B22 - Gold,CYGNETT Smart LED Lightstrip Extension (1m),LENOVO,ELECTRONICS,0
993,2 x Lenovo 4.5W Smart Filament Bulb C35 E14 - ...,APPLE iPhone 12 64GB - Green,LENOVO,ELECTRONICS,0
994,Electrolux Silent Performer Bagged Origin Vacu...,RUSSELL HOBBS Russell Hobbs Brooklyn Kettle - ...,ELECTROLUX,APPLIANCES,0
995,Shark Navigator Self-Cleaning Brushroll Pet Up...,RUSSELL HOBBS Russell Hobbs Steam Power Steam ...,SHARK,APPLIANCES,0


In [ ]:
df_output.head()

,external_name,internal_name,BrandName (BP),department,match
0,Garmin Vivosmart 4 Fitness Tracker S/M - Grey/...,GARMIN Vivosmart 4 Activity Tracker Grey Rose ...,GARMIN,ELECTRONICS,1
1,"Garmin 5"" Drive 52 & Live Traffic GPS",GARMIN Drive 52 GPS,GARMIN,ELECTRONICS,1
2,Garmin 45mm Instinct Bluetooth GPS Sport Watch...,GARMIN Instinct Sports Watch - Tundra,GARMIN,ELECTRONICS,1
3,Garmin 30.4mm Forerunner 245 Music Smartwatch ...,GARMIN Forerunner 245 Music Black,GARMIN,ELECTRONICS,1
4,Garmin 30.4mm Forerunner 245 Music Fitness Sma...,GARMIN Garmin Forerunner 245 Music Smart Watch...,GARMIN,ELECTRONICS,1


In [ ]:
df_output.shape

(10956, 5)

In [ ]:
df_output.match.value_counts()

0    9943
1    1013
Name: match, dtype: int64

In [ ]:
df_output.to_csv('Brand_product_matching_synthetic.csv', index=False)

In [ ]:
ls

Brand_product_matching_synthetic.csv  drive/  Invigor/  sample_data/


In [ ]:
import shutil

In [ ]:
shutil.move('/content/Brand_product_matching_synthetic.csv','/content/drive/MyDrive')

'/content/drive/MyDrive/Brand_product_matching_synthetic.csv'